In [38]:
import pandas as pd
from sqlalchemy import create_engine
from config import postgres

### Store CSV into DataFrame

In [39]:
csv_file = "Resources/top10s.csv"
spotify_df = pd.read_csv(csv_file,encoding='cp1252')
spotify_df.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [40]:
#Rename columns to align to pgAdmin table
spotify_df = spotify_df.rename(columns = {"title":"track_title",
                                         "top genre": "top_genre",
                                          "dB": "db",
                                         "Unnamed: 0": "ref"})

spotify_df.head()


,ref,track_title,artist,top_genre,year,bpm,nrgy,dnce,db,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [10]:
#Check artist names for importing issues - spot check Beyonce
print(spotify_df['artist'].unique())

['Train' 'Eminem' 'Kesha' 'Lady Gaga' 'Bruno Mars' 'Justin Bieber'
 'Taio Cruz' 'OneRepublic' 'Alicia Keys' 'Rihanna' 'Flo Rida'
 'Mike Posner' 'Far East Movement' 'Usher' 'Sean Kingston'
 'The Black Eyed Peas' 'Adam Lambert' 'Maroon 5' 'Neon Trees'
 'Selena Gomez & The Scene' 'Enrique Iglesias' 'Katy Perry'
 'Britney Spears' '3OH!3' 'David Guetta' 'Christina Aguilera'
 'Florence + The Machine' 'Shakira' 'Tinie Tempah' 'T.I.' 'Martin Solveig'
 'Christina Perri' 'Adele' 'Pitbull' 'Beyoncé' 'Hot Chelle Rae'
 'Avril Lavigne' 'Kanye West' 'LMFAO' 'Jessie J' 'Jennifer Lopez'
 'Chris Brown' 'Sleeping At Last' 'Nicki Minaj' 'P!nk' 'Coldplay'
 'One Direction' 'Taylor Swift' 'Carly Rae Jepsen' 'Kelly Clarkson'
 'Owl City' 'The Wanted' 'fun.' 'Ellie Goulding' 'Gym Class Heroes'
 'Avicii' 'The Script' 'Miley Cyrus' 'Swedish House Mafia' 'Daft Punk'
 'James Arthur' 'Robin Thicke' 'Demi Lovato' 'Zedd' 'Lana Del Rey'
 'Calvin Harris' 'Drake' 'Sara Bareilles' 'Macklemore & Ryan Lewis'
 'Labrinth' 'Ne

### Connect to local database

In [42]:
rds_connection_string = f"postgres:{postgres}localhost:5432/music-masters_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [43]:
engine.table_names()

['spotify_toptracks']

### Use pandas to load csv converted DataFrame into database

In [28]:
spotify_df.to_sql(name='spotify_toptracks', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the spotify table
* NOTE: can also check using pgAdmin

In [29]:
pd.read_sql_query('select * from spotify_toptracks', con=engine).head()

,ref,track_title,artist,top_genre,year,bpm,nrgy,dnce,db,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78
